In [64]:
import pandas as pd
import numpy as np
import nltk
import pickle
import os
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Embedding, LSTM, Dense, Dropout
from tensorflow.keras.optimizers import Adam
from AttentionLayer import Attention

In [65]:
nltk.data.path.append("C:/Users/sakti/nltkdata") 

In [66]:
os.makedirs("model", exist_ok=True)
os.makedirs("tokenizer", exist_ok=True)

In [67]:
nltk.download('stopwords')
nltk.download('punkt')
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\sakti\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\sakti\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [68]:
df = pd.read_csv("C:/Users/sakti/Downloads/IMDB Dataset/IMDB Dataset.csv")
df['sentiment'] = df['sentiment'].map({'positive': 1, 'negative': 0})

In [69]:
import nltk

nltk.download('stopwords', download_dir='C:/Users/sakti/nltk_data')
nltk.download('punkt', download_dir='C:/Users/sakti/nltk_data')
nltk.download('wordnet', download_dir='C:/Users/sakti/nltk_data')
nltk.download('omw-1.4', download_dir='C:/Users/sakti/nltk_data')


[nltk_data] Downloading package stopwords to
[nltk_data]     C:/Users/sakti/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to C:/Users/sakti/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to C:/Users/sakti/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to C:/Users/sakti/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


True

In [70]:
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
import string

stop_words = set(stopwords.words('english'))
lemmatizer = WordNetLemmatizer()

def preprocess_text(text):
    # Lowercase
    text = text.lower()
    
    # Remove punctuation
    text = text.translate(str.maketrans('', '', string.punctuation))  
    
    # Tokenize
    tokens = word_tokenize(text)
    
    # Remove stopwords
    filtered = [word for word in tokens if word not in stop_words]
    
    # Lemmatize
    lemmatized = [lemmatizer.lemmatize(word) for word in filtered]
    
    return ' '.join(lemmatized)


In [71]:
import nltk
nltk.data.path.append(r"C:\nltk_data")
nltk.download('punkt', download_dir=r"C:\nltk_data")
nltk.download('stopwords', download_dir=r"C:\nltk_data")

from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords

# def preprocess_text(text):
#     stop_words = set(stopwords.words('english'))
#     tokens = word_tokenize(text.lower())
#     tokens = [word for word in tokens if word.isalpha() and word not in stop_words]
#     return " ".join(tokens)


[nltk_data] Downloading package punkt to C:\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to C:\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [72]:
df['cleaned'] = df['review'].map(lambda x: x.lower().strip())
# df.head()


In [73]:
MAX_VOCAB = 10000
MAX_LEN = 200

In [74]:
tokenizer = Tokenizer(num_words=MAX_VOCAB)
tokenizer.fit_on_texts(df['cleaned'])
X = pad_sequences(tokenizer.texts_to_sequences(df['cleaned']), maxlen=MAX_LEN)
y = df['sentiment'].values

In [75]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [76]:
input_layer = Input(shape=(MAX_LEN,))
embedding = Embedding(input_dim=MAX_VOCAB, output_dim=128)(input_layer)
lstm = LSTM(64, return_sequences=True)(embedding)
attention = Attention()(lstm)
dropout = Dropout(0.5)(attention)
output = Dense(1, activation='sigmoid')(dropout)

In [77]:
model = Model(inputs=input_layer, outputs=output)
model.compile(loss='binary_crossentropy', optimizer=Adam(0.001), metrics=['accuracy'])
model.summary()

Model: "functional_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_1 (InputLayer)      │ (None, 200)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ embedding_1 (Embedding)         │ (None, 200, 128)       │     1,280,000 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_1 (LSTM)                   │ (None, 200, 64)        │        49,408 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ attention_1 (Attention)         │ (None, 64)             │           264 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 1)              │            65 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 1,329,737 (5.07 MB)

 Trainable params: 1,329,737 (5.07 MB)

 Non-trainable params: 0 (0.00 B)

In [78]:
model.fit(X_train, y_train, epochs=5, batch_size=128, validation_split=0.1)

Epoch 1/5
282/282 ━━━━━━━━━━━━━━━━━━━━ 57s 193ms/step - accuracy: 0.7139 - loss: 0.5327 - val_accuracy: 0.8777 - val_loss: 0.2906
Epoch 2/5
282/282 ━━━━━━━━━━━━━━━━━━━━ 52s 183ms/step - accuracy: 0.9073 - loss: 0.2392 - val_accuracy: 0.8848 - val_loss: 0.2784
Epoch 3/5
282/282 ━━━━━━━━━━━━━━━━━━━━ 51s 179ms/step - accuracy: 0.9335 - loss: 0.1764 - val_accuracy: 0.8850 - val_loss: 0.2944
Epoch 4/5
282/282 ━━━━━━━━━━━━━━━━━━━━ 52s 186ms/step - accuracy: 0.9495 - loss: 0.1406 - val_accuracy: 0.8860 - val_loss: 0.3203
Epoch 5/5
282/282 ━━━━━━━━━━━━━━━━━━━━ 54s 193ms/step - accuracy: 0.9651 - loss: 0.1038 - val_accuracy: 0.8815 - val_loss: 0.3623


In [79]:
model.save("model/sentiment_model.h5")
with open("tokenizer/tokenizer.pkl", "wb") as f:
    pickle.dump(tokenizer, f)

In [80]:
print("✅ Model and tokenizer saved successfully!")

✅ Model and tokenizer saved successfully!


In [ ]:
# Improved IMDB Sentiment Training Script (with punkt fallback)

import pandas as pd
import numpy as np
import nltk
import pickle
import os
import re
from sklearn.model_selection import train_test_split
from sklearn.utils import class_weight
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Embedding, LSTM, Dense, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
from AttentionLayer import Attention

# Ensure folders exist
os.makedirs("model", exist_ok=True)
os.makedirs("tokenizer", exist_ok=True)

# Download NLTK resources
nltk.download('stopwords')
nltk.download('punkt')
from nltk.corpus import stopwords

# Load dataset
df = pd.read_csv("IMDB Dataset.csv")
df['sentiment'] = df['sentiment'].map({'positive': 1, 'negative': 0})

# Check class distribution
print("Class distribution:")
print(df['sentiment'].value_counts())

# Fallback preprocessing (in case punkt errors)
def preprocess_text(text):
    stop_words = set(stopwords.words('english'))
    tokens = re.findall(r'\b\w+\b', text.lower())
    tokens = [word for word in tokens if word not in stop_words]
    return " ".join(tokens)

df['cleaned'] = df['review'].apply(preprocess_text)

# Tokenization
MAX_VOCAB = 10000
MAX_LEN = 200
tokenizer = Tokenizer(num_words=MAX_VOCAB)
tokenizer.fit_on_texts(df['cleaned'])
X = pad_sequences(tokenizer.texts_to_sequences(df['cleaned']), maxlen=MAX_LEN)
y = df['sentiment'].values

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Class weights
class_weights = class_weight.compute_class_weight(class_weight='balanced', classes=np.unique(y_train), y=y_train)
class_weights_dict = dict(enumerate(class_weights))
print("Class weights:", class_weights_dict)

# Model architecture
input_layer = Input(shape=(MAX_LEN,))
embedding = Embedding(input_dim=MAX_VOCAB, output_dim=128)(input_layer)
lstm = LSTM(64, return_sequences=True)(embedding)
attention = Attention()(lstm)
dropout = Dropout(0.5)(attention)
output = Dense(1, activation='sigmoid')(dropout)

model = Model(inputs=input_layer, outputs=output)
model.compile(loss='binary_crossentropy', optimizer=Adam(0.001), metrics=['accuracy'])
model.summary()

# Training with early stopping
early_stop = EarlyStopping(patience=2, restore_best_weights=True)
model.fit(X_train, y_train, 
          epochs=10, 
          batch_size=128, 
          validation_split=0.1, 
          class_weight=class_weights_dict, 
          callbacks=[early_stop])

# Evaluate on test set
loss, acc = model.evaluate(X_test, y_test)
print("Test Accuracy:", acc)

# Save model and tokenizer
model.save("model/sentiment_model.h5")
with open("tokenizer/tokenizer.pkl", "wb") as f:
    pickle.dump(tokenizer, f)

print("\n✅ Model and tokenizer saved successfully!")

# Test predictions
sample_reviews = [
    "I love this movie.",]

for review in sample_reviews:
    cleaned = preprocess_text(review)
    seq = tokenizer.texts_to_sequences([cleaned])
    padded = pad_sequences(seq, maxlen=MAX_LEN)
    pred = model.predict(padded)[0][0]
    label = "Positive 😊" if pred >= 0.55 else "Negative 😞" 
    print(f"\nReview: {review}\nPrediction: {label} (score={pred:.2f})")


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\sakti\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\sakti\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


Class distribution:
sentiment
1    25000
0    25000
Name: count, dtype: int64
Class weights: {0: 0.9980537950995558, 1: 1.0019538099293623}


Model: "functional_4"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_3 (InputLayer)      │ (None, 200)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ embedding_3 (Embedding)         │ (None, 200, 128)       │     1,280,000 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_3 (LSTM)                   │ (None, 200, 64)        │        49,408 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ attention_3 (Attention)         │ (None, 64)             │           264 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_3 (Dropout)             │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 1)              │            65 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 1,329,737 (5.07 MB)

 Trainable params: 1,329,737 (5.07 MB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/10
282/282 ━━━━━━━━━━━━━━━━━━━━ 58s 193ms/step - accuracy: 0.6913 - loss: 0.5495 - val_accuracy: 0.8840 - val_loss: 0.2805
Epoch 2/10
282/282 ━━━━━━━━━━━━━━━━━━━━ 48s 169ms/step - accuracy: 0.9190 - loss: 0.2217 - val_accuracy: 0.8867 - val_loss: 0.2759
Epoch 3/10
282/282 ━━━━━━━━━━━━━━━━━━━━ 51s 180ms/step - accuracy: 0.9412 - loss: 0.1647 - val_accuracy: 0.8830 - val_loss: 0.2977
Epoch 4/10
282/282 ━━━━━━━━━━━━━━━━━━━━ 47s 166ms/step - accuracy: 0.9578 - loss: 0.1242 - val_accuracy: 0.8788 - val_loss: 0.3576
313/313 ━━━━━━━━━━━━━━━━━━━━ 5s 17ms/step - accuracy: 0.8829 - loss: 0.2724


Test Accuracy: 0.8848999738693237

✅ Model and tokenizer saved successfully!
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 226ms/step

Review: I hated this movie. It was boring and too long.
Prediction: Neutral 😐 (score=0.50)


In [ ]:
# Test predictions



1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 75ms/step

Review: I love this movie.
Prediction: Positive 😊 (score=0.59)
